<a href="https://colab.research.google.com/github/Aswin-Cheerngodan/Sentimen-analysis-colab/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split


Data Collection

In [19]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [20]:
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [21]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [22]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [23]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
    zip_ref.extractall()

In [24]:
data = pd.read_csv('IMDB Dataset.csv')

In [25]:
data.shape

(50000, 2)

In [26]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [27]:
data.sentiment.value_counts()

,count
sentiment,
positive,25000
negative,25000


In [28]:
data.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)

<ipython-input-28-a971eddbce1c>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment':{'positive':1,'negative':0}},inplace=True)


In [29]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [30]:
!pip install contractions
nltk.download('all',quiet=True)

True

In [31]:
import nltk
import re
import contractions
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

Data Cleaning

In [32]:
def remove_noise(text):
    #removing html tags
    text = BeautifulSoup(text,'html.parser').get_text()
    #remove urls
    text = re.sub(r'http\S+|www\S+|https\S+','',text,flags=re.MULTILINE)
    #expanding contradictions
    text = contractions.fix(text)
    #remove the punctuation like things
    text = re.sub(r'[^a-zA-Z\s]','',text)
    #to lowercase
    text = text.lower()
    #remove stopwords
    stop_words = set(stopwords.words('english'))

    words = text.split()

    filtered_words = [word for word in words if word not in stop_words]

    return ' '.join(filtered_words)

In [33]:
data['review_cl'] = data['review'].apply(remove_noise)

<ipython-input-32-3782274f71a7>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text,'html.parser').get_text()


In [34]:
data.head()

,review,sentiment,review_cl
0,One of the other reviewers has mentioned that ...,1,one reviewers mentioned watching oz episode ho...
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,0,basically family little boy jake thinks zombie...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love time money visually stunni...


Lemmaization

In [35]:
data['review'] = data['review_cl']
data.drop('review_cl',axis=1,inplace=True)
data.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter matteis love time money visually stunni...,1


Lemmaization

In [41]:
from nltk.stem import WordNetLemmatizer

def lemmaize(text):
    Lemmatizer = WordNetLemmatizer()
    words = text.split()
    return " ".join([Lemmatizer.lemmatize(word) for word in words])

In [42]:
data['review_lm'] = data['review'].apply(lemmaize)
data.head()

,review,sentiment,review_lm
0,one reviewers mentioned watching oz episode ho...,1,one reviewer mentioned watching oz episode hoo...
1,wonderful little production filming technique ...,1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...,1,thought wonderful way spend time hot summer we...
3,basically family little boy jake thinks zombie...,0,basically family little boy jake think zombie ...
4,petter matteis love time money visually stunni...,1,petter matteis love time money visually stunni...


In [43]:
data.drop('review',axis=1,inplace=True)
data.head()

,sentiment,review_lm
0,1,one reviewer mentioned watching oz episode hoo...
1,1,wonderful little production filming technique ...
2,1,thought wonderful way spend time hot summer we...
3,0,basically family little boy jake think zombie ...
4,1,petter matteis love time money visually stunni...


Split data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

Train_data,test_data = train_test_split(data,test_size=0.2,random_state=42)

